In [1]:
import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import random
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("=== COMPREHENSIVE DES SIMULATION FOR NHS CATARACT SERVICES ===")

# Load real data for calibration
surgery_data = pd.read_csv('surgery_data_cleaned_for_modeling.csv')
surgery_data['Start_clock_date'] = pd.to_datetime(surgery_data['Start_clock_date'])
surgery_data['End_clock_date'] = pd.to_datetime(surgery_data['End_clock_date'])

print(f"Historical data loaded: {len(surgery_data)} procedures")

# Real provider constraints from our previous analysis
PROVIDER_REAL_DATA = {
    'CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST': {
        'theatres': 2,
        'daily_capacity': 9.6,
        'current_waiting_list': 800,
        'actual_utilization': 0.95,
        'cases_per_session': 6,
        'operating_hours_start': 8,  # 8 AM
        'operating_hours_end': 18,   # 6 PM
        'operating_days': [1, 2, 3, 4, 5],  # Mon-Fri
        'provider_type': 'NHS',
        'historical_procedures': 2014
    },
    'NORTH WEST ANGLIA NHS FOUNDATION TRUST': {
        'theatres': 4,
        'daily_capacity': 16.8,
        'current_waiting_list': 1200,
        'actual_utilization': 0.92,
        'cases_per_session': 7,
        'operating_hours_start': 8,
        'operating_hours_end': 17,   # 5 PM
        'operating_days': [1, 2, 3, 4, 5],
        'provider_type': 'NHS',
        'historical_procedures': 2219
    },
    'FITZWILLIAM HOSPITAL': {
        'theatres': 5,
        'daily_capacity': 16.0,
        'current_waiting_list': 100,
        'actual_utilization': 0.75,
        'cases_per_session': 8,
        'operating_hours_start': 8,
        'operating_hours_end': 18,
        'operating_days': [1, 2, 3, 4, 5, 6],  # Mon-Sat
        'provider_type': 'Independent',
        'historical_procedures': 152
    },
    'ANGLIA COMMUNITY EYE SERVICE LTD': {
        'theatres': 3,
        'daily_capacity': 17.6,
        'current_waiting_list': 300,
        'actual_utilization': 0.88,
        'cases_per_session': 8,
        'operating_hours_start': 9,
        'operating_hours_end': 17,
        'operating_days': [1, 2, 3, 4, 5],
        'provider_type': 'Independent',
        'historical_procedures': 1793
    },
    'SPAMEDICA PETERBOROUGH': {
        'theatres': 2,
        'daily_capacity': 17.6,
        'current_waiting_list': 200,
        'actual_utilization': 0.85,
        'cases_per_session': 10,
        'operating_hours_start': 8,
        'operating_hours_end': 17,
        'operating_days': [1, 2, 3, 4, 5, 6],
        'provider_type': 'Independent',
        'historical_procedures': 1777
    },
    'SPAMEDICA BEDFORD': {
        'theatres': 2,
        'daily_capacity': 19.8,
        'current_waiting_list': 150,
        'actual_utilization': 0.82,
        'cases_per_session': 9,
        'operating_hours_start': 8,
        'operating_hours_end': 17,
        'operating_days': [1, 2, 3, 4, 5],
        'provider_type': 'Independent',
        'historical_procedures': 468
    }
}

# Calculate total system arrival rate from historical data
total_historical_procedures = sum(p['historical_procedures'] for p in PROVIDER_REAL_DATA.values())
simulation_years = 2
total_simulation_days = simulation_years * 365
system_arrival_rate = total_historical_procedures / (simulation_years * 250)  # 250 working days per year

print(f"System parameters:")
print(f"  Total historical procedures: {total_historical_procedures}")
print(f"  System arrival rate: {system_arrival_rate:.2f} patients/day")
print(f"  Simulation period: {simulation_years} years ({total_simulation_days} days)")

class Patient:
    """Patient entity with realistic NHS characteristics"""
    def __init__(self, patient_id, arrival_time, priority, hrg_code, referral_source=None):
        self.id = patient_id
        self.arrival_time = arrival_time
        self.priority = priority
        self.hrg_code = hrg_code
        self.referral_source = referral_source
        
        # Patient journey tracking
        self.assigned_provider = None
        self.referral_date = arrival_time
        self.first_assessment_date = None
        self.surgery_booking_date = None
        self.surgery_date = None
        self.discharge_date = None
        
        # Time metrics
        self.referral_to_assessment_days = 0
        self.assessment_to_surgery_days = 0
        self.total_pathway_days = 0
        self.actual_waiting_time = 0
        
        # Clinical attributes
        self.complexity_score = self.assign_complexity()
        self.expected_operating_time = self.calculate_operating_time()
        
    def assign_complexity(self):
        """Assign complexity based on HRG code"""
        complexity_map = {
            'BZ33Z': 1,  # Minor
            'BZ32B': 2, 'BZ32A': 3,  # Intermediate
            'BZ34C': 2, 'BZ34B': 3, 'BZ34A': 4,  # Phaco
            'BZ30A': 5, 'BZ30B': 5,  # Complex
            'BZ31A': 6, 'BZ31B': 5   # Very Major
        }
        return complexity_map.get(self.hrg_code, 3)
    
    def calculate_operating_time(self):
        """Calculate expected operating time including prep and turnover"""
        base_times = {
            'BZ33Z': 25, 'BZ32B': 35, 'BZ32A': 40,
            'BZ34C': 30, 'BZ34B': 35, 'BZ34A': 45,
            'BZ30A': 60, 'BZ30B': 55, 'BZ31A': 80, 'BZ31B': 70
        }
        base_time = base_times.get(self.hrg_code, 35)
        
        # Add prep time (15 min) + cleaning (15 min) + setup (10 min)
        total_time = base_time + 40
        
        # Add variability
        return max(20, np.random.normal(total_time, total_time * 0.2))

class NHSProvider:
    """NHS Provider with realistic scheduling and capacity constraints"""
    def __init__(self, env, name, config):
        self.env = env
        self.name = name
        self.config = config
        
        # Theatre resources (only available during operating hours)
        self.theatres = simpy.Resource(env, capacity=config['theatres'])
        
        # Initialize with existing waiting list
        self.waiting_list = []
        self.create_initial_waiting_list()
        
        # Statistics tracking
        self.patients_treated = 0
        self.total_operating_time = 0
        self.daily_completions = []
        self.utilization_history = []
        self.queue_length_history = []
        self.wait_time_history = []
        
        # Start processes
        env.process(self.manage_daily_operations())
        env.process(self.process_waiting_list())
        env.process(self.monitor_performance())
        
    def create_initial_waiting_list(self):
        """Create initial waiting list based on current backlog"""
        for i in range(self.config['current_waiting_list']):
            # Create patients with varying wait times (some have been waiting longer)
            days_already_waiting = np.random.exponential(30)  # Average 30 days already waited
            arrival_time = self.env.now - days_already_waiting/365  # Convert to simulation time
            
            # Generate realistic patient
            hrg_probs = [0.38, 0.31, 0.10, 0.08, 0.05, 0.04, 0.02, 0.01, 0.01]
            hrg_codes = ['BZ34C', 'BZ34B', 'BZ33Z', 'BZ34A', 'BZ31A', 'BZ31B', 'BZ30A', 'BZ32B', 'BZ32A']
            hrg_code = np.random.choice(hrg_codes, p=hrg_probs)
            
            patient = Patient(
                patient_id=f"INIT_{self.name[:3]}_{i}",
                arrival_time=arrival_time,
                priority=np.random.choice([1, 2], p=[0.95, 0.05]),  # Mostly routine
                hrg_code=hrg_code
            )
            patient.assigned_provider = self.name
            self.waiting_list.append(patient)
        
        print(f"  {self.name[:25]}: Created initial waiting list of {len(self.waiting_list)} patients")
    
    def is_operating_day(self):
        """Check if today is an operating day"""
        current_day = int(self.env.now) % 7  # 0=Sunday, 1=Monday, etc.
        return current_day in self.config['operating_days']
    
    def is_operating_hours(self):
        """Check if current time is within operating hours"""
        if not self.is_operating_day():
            return False
        
        current_hour = (self.env.now % 1) * 24  # Hour of day
        return self.config['operating_hours_start'] <= current_hour <= self.config['operating_hours_end']
    
    def manage_daily_operations(self):
        """Manage daily theatre operations and scheduling"""
        while True:
            if self.is_operating_day():
                # Start of operating day
                yield self.env.timeout(self.config['operating_hours_start'] / 24)
                
                # Schedule theatre sessions
                operating_duration = (self.config['operating_hours_end'] - self.config['operating_hours_start']) / 24
                yield self.env.timeout(operating_duration)
                
                # End of day - record statistics
                daily_completions = len([p for p in self.wait_time_history 
                                       if int(self.env.now - 1) <= int(p) < int(self.env.now)])
                self.daily_completions.append((int(self.env.now), daily_completions))
                
                # Rest of day
                rest_duration = (24 - self.config['operating_hours_end']) / 24
                yield self.env.timeout(rest_duration)
            else:
                # Non-operating day
                yield self.env.timeout(1.0)
    
    def process_waiting_list(self):
        """Process patients from waiting list when theatres are available"""
        while True:
            if self.waiting_list and self.is_operating_hours():
                # Get next patient (FIFO with priority)
                urgent_patients = [p for p in self.waiting_list if p.priority == 2]  # Urgent
                if urgent_patients:
                    patient = urgent_patients[0]
                    self.waiting_list.remove(patient)
                else:
                    patient = self.waiting_list.pop(0)  # First in queue
                
                # Process patient
                self.env.process(self.treat_patient(patient))
                
            yield self.env.timeout(0.1)  # Check every 2.4 hours
    
    def treat_patient(self, patient):
        """Treat patient in theatre"""
        # Request theatre
        with self.theatres.request() as request:
            yield request
            
            # Patient enters theatre
            patient.surgery_date = self.env.now
            patient.actual_waiting_time = (patient.surgery_date - patient.arrival_time) * 365  # Convert to days
            
            # Surgery duration (convert minutes to simulation time)
            surgery_time_days = patient.expected_operating_time / (60 * 24)
            
            # Perform surgery
            yield self.env.timeout(surgery_time_days)
            
            # Complete surgery
            patient.discharge_date = self.env.now
            patient.total_pathway_days = (patient.discharge_date - patient.arrival_time) * 365
            
            # Record statistics
            self.patients_treated += 1
            self.total_operating_time += patient.expected_operating_time
            self.wait_time_history.append(patient.actual_waiting_time)
            
    def add_patient_to_list(self, patient):
        """Add new patient to waiting list"""
        patient.assigned_provider = self.name
        self.waiting_list.append(patient)
    
    def monitor_performance(self):
        """Monitor provider performance metrics"""
        while True:
            current_utilization = len(self.theatres.users) / self.config['theatres']
            queue_length = len(self.waiting_list)
            
            self.utilization_history.append((self.env.now, current_utilization))
            self.queue_length_history.append((self.env.now, queue_length))
            
            yield self.env.timeout(0.5)  # Monitor twice daily

class NHSCataractSystem:
    """Complete NHS Cataract System Simulation"""
    def __init__(self, env, system_config):
        self.env = env
        self.system_config = system_config
        
        # Create providers
        self.providers = {}
        for name, config in PROVIDER_REAL_DATA.items():
            self.providers[name] = NHSProvider(env, name, config)
        
        # System-wide statistics
        self.total_patients_generated = 0
        self.total_patients_treated = 0
        self.system_performance = []
        
        # Start system processes
        env.process(self.generate_patient_arrivals())
        env.process(self.monitor_system_performance())
    
    def generate_patient_arrivals(self):
        """Generate patient arrivals based on historical patterns"""
        while True:
            # Inter-arrival time based on system arrival rate
            inter_arrival = np.random.exponential(1 / system_arrival_rate)
            yield self.env.timeout(inter_arrival / 365)  # Convert to simulation time
            
            # Create new patient
            self.total_patients_generated += 1
            
            # Realistic patient characteristics
            hrg_probs = [0.38, 0.31, 0.10, 0.08, 0.05, 0.04, 0.02, 0.01, 0.01]
            hrg_codes = ['BZ34C', 'BZ34B', 'BZ33Z', 'BZ34A', 'BZ31A', 'BZ31B', 'BZ30A', 'BZ32B', 'BZ32A']
            hrg_code = np.random.choice(hrg_codes, p=hrg_probs)
            
            patient = Patient(
                patient_id=f"NEW_{self.total_patients_generated}",
                arrival_time=self.env.now,
                priority=np.random.choice([1, 2, 3], p=[0.95, 0.04, 0.01]),
                hrg_code=hrg_code
            )
            
            # Route patient to provider
            if self.system_config['consolidated']:
                provider = self.route_patient_consolidated(patient)
            else:
                provider = self.route_patient_fragmented(patient)
            
            provider.add_patient_to_list(patient)
    
    def route_patient_fragmented(self, patient):
        """Route patient in fragmented system (current NHS)"""
        # Weighted random selection based on historical volume
        provider_weights = [config['historical_procedures'] for config in PROVIDER_REAL_DATA.values()]
        provider_names = list(PROVIDER_REAL_DATA.keys())
        
        selected_name = np.random.choice(provider_names, p=np.array(provider_weights) / sum(provider_weights))
        return self.providers[selected_name]
    
    def route_patient_consolidated(self, patient):
        """Route patient in consolidated system (optimal routing)"""
        # Intelligent routing based on:
        # 1. Current queue lengths
        # 2. Provider capacity
        # 3. Patient complexity
        
        suitable_providers = list(self.providers.values())
        
        # For complex cases, prefer larger providers
        if patient.complexity_score >= 5:
            suitable_providers = [p for p in suitable_providers if p.config['theatres'] >= 3]
            if not suitable_providers:
                suitable_providers = list(self.providers.values())
        
        # Choose provider with lowest queue-to-capacity ratio
        def load_metric(provider):
            queue_load = len(provider.waiting_list) / provider.config['daily_capacity']
            utilization_penalty = provider.config['actual_utilization']
            return queue_load * (1 + utilization_penalty)
        
        return min(suitable_providers, key=load_metric)
    
    def monitor_system_performance(self):
        """Monitor overall system performance"""
        while True:
            yield self.env.timeout(7)  # Weekly monitoring
            
            # Calculate system metrics
            total_waiting = sum(len(p.waiting_list) for p in self.providers.values())
            total_treated = sum(p.patients_treated for p in self.providers.values())
            avg_utilization = np.mean([p.config['actual_utilization'] for p in self.providers.values()])
            
            self.system_performance.append({
                'week': int(self.env.now * 365 / 7),
                'total_waiting': total_waiting,
                'total_treated': total_treated,
                'avg_utilization': avg_utilization
            })

def run_nhs_simulation(scenario='fragmented', simulation_time=730):  # 2 years
    """Run NHS cataract system simulation"""
    print(f"\n🔄 Running {scenario.upper()} NHS simulation...")
    print(f"   Simulation time: {simulation_time} days ({simulation_time/365:.1f} years)")
    
    # Create simulation environment
    env = simpy.Environment()
    
    # System configuration
    system_config = {
        'consolidated': scenario == 'consolidated',
        'simulation_time': simulation_time
    }
    
    # Create NHS system
    nhs_system = NHSCataractSystem(env, system_config)
    
    # Run simulation
    print("   Starting simulation...")
    env.run(until=simulation_time / 365)  # Convert days to simulation time
    
    # Collect results
    results = {
        'scenario': scenario,
        'patients_generated': nhs_system.total_patients_generated,
        'system_performance': nhs_system.system_performance,
        'providers': {}
    }
    
    # Collect provider-specific results
    for name, provider in nhs_system.providers.items():
        results['providers'][name] = {
            'patients_treated': provider.patients_treated,
            'current_waiting_list': len(provider.waiting_list),
            'wait_times': provider.wait_time_history,
            'utilization_history': provider.utilization_history,
            'queue_history': provider.queue_length_history
        }
    
    print(f"✅ {scenario.capitalize()} simulation completed:")
    print(f"   Patients generated: {nhs_system.total_patients_generated}")
    print(f"   Total patients treated: {sum(p.patients_treated for p in nhs_system.providers.values())}")
    print(f"   Remaining in system: {sum(len(p.waiting_list) for p in nhs_system.providers.values())}")
    
    return results

def analyze_simulation_results(fragmented_results, consolidated_results):
    """Analyze and compare simulation results"""
    print("\n=== NHS SIMULATION RESULTS ANALYSIS ===")
    
    # Provider-level analysis
    provider_comparison = []
    
    for provider_name in fragmented_results['providers'].keys():
        frag_data = fragmented_results['providers'][provider_name]
        cons_data = consolidated_results['providers'][provider_name]
        
        frag_avg_wait = np.mean(frag_data['wait_times']) if frag_data['wait_times'] else 0
        cons_avg_wait = np.mean(cons_data['wait_times']) if cons_data['wait_times'] else 0
        
        provider_comparison.append({
            'Provider': provider_name[:25],
            'Fragmented_Treated': frag_data['patients_treated'],
            'Consolidated_Treated': cons_data['patients_treated'],
            'Fragmented_Waiting_List': frag_data['current_waiting_list'],
            'Consolidated_Waiting_List': cons_data['current_waiting_list'],
            'Fragmented_Avg_Wait_Days': frag_avg_wait,
            'Consolidated_Avg_Wait_Days': cons_avg_wait,
            'Wait_Time_Improvement': ((frag_avg_wait - cons_avg_wait) / frag_avg_wait * 100) if frag_avg_wait > 0 else 0
        })
    
    comparison_df = pd.DataFrame(provider_comparison)
    
    print("\n📊 PROVIDER-LEVEL COMPARISON:")
    display_cols = ['Provider', 'Fragmented_Avg_Wait_Days', 'Consolidated_Avg_Wait_Days', 
                   'Wait_Time_Improvement', 'Fragmented_Waiting_List', 'Consolidated_Waiting_List']
    print(comparison_df[display_cols].round(2))
    
    # System-level metrics
    total_frag_treated = sum(p['patients_treated'] for p in fragmented_results['providers'].values())
    total_cons_treated = sum(p['patients_treated'] for p in consolidated_results['providers'].values())
    
    total_frag_waiting = sum(p['current_waiting_list'] for p in fragmented_results['providers'].values())
    total_cons_waiting = sum(p['current_waiting_list'] for p in consolidated_results['providers'].values())
    
    print(f"\n🎯 SYSTEM-LEVEL IMPROVEMENTS:")
    print(f"  Throughput improvement: {((total_cons_treated - total_frag_treated) / total_frag_treated * 100):.1f}%")
    print(f"  Waiting list reduction: {((total_frag_waiting - total_cons_waiting) / total_frag_waiting * 100):.1f}%")
    print(f"  Average wait time improvement: {comparison_df['Wait_Time_Improvement'].mean():.1f}%")
    
    # Create visualizations
    create_simulation_visualizations(comparison_df, fragmented_results, consolidated_results)
    
    # Save results
    comparison_df.to_csv('nhs_des_simulation_results.csv', index=False)
    print(f"\n✅ Simulation results saved to 'nhs_des_simulation_results.csv'")
    
    return comparison_df

def create_simulation_visualizations(comparison_df, frag_results, cons_results):
    """Create comprehensive visualizations of simulation results"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))
    
    # 1. Wait time comparison
    providers_short = [p[:12] for p in comparison_df['Provider']]
    x = np.arange(len(providers_short))
    width = 0.35
    
    ax1.bar(x - width/2, comparison_df['Fragmented_Avg_Wait_Days'], width, 
            label='Fragmented', color='red', alpha=0.7)
    ax1.bar(x + width/2, comparison_df['Consolidated_Avg_Wait_Days'], width, 
            label='Consolidated', color='blue', alpha=0.7)
    ax1.set_xlabel('Provider')
    ax1.set_ylabel('Average Wait Time (days)')
    ax1.set_title('Wait Time Comparison: Fragmented vs Consolidated')
    ax1.set_xticks(x)
    ax1.set_xticklabels(providers_short, rotation=45)
    ax1.legend()
    
    # 2. Waiting list sizes
    ax2.bar(x - width/2, comparison_df['Fragmented_Waiting_List'], width, 
            label='Fragmented', color='red', alpha=0.7)
    ax2.bar(x + width/2, comparison_df['Consolidated_Waiting_List'], width, 
            label='Consolidated', color='blue', alpha=0.7)
    ax2.set_xlabel('Provider')
    ax2.set_ylabel('Final Waiting List Size')
    ax2.set_title('Waiting List Sizes: Fragmented vs Consolidated')
    ax2.set_xticks(x)
    ax2.set_xticklabels(providers_short, rotation=45)
    ax2.legend()
    
    # 3. Improvement percentages
    colors = ['green' if imp > 0 else 'red' for imp in comparison_df['Wait_Time_Improvement']]
    ax3.bar(providers_short, comparison_df['Wait_Time_Improvement'], color=colors, alpha=0.7)
    ax3.set_xlabel('Provider')
    ax3.set_ylabel('Wait Time Improvement (%)')
    ax3.set_title('Wait Time Improvement by Provider')
    ax3.tick_params(axis='x', rotation=45)
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # 4. Throughput comparison
    ax4.bar(x - width/2, comparison_df['Fragmented_Treated'], width, 
            label='Fragmented', color='red', alpha=0.7)
    ax4.bar(x + width/2, comparison_df['Consolidated_Treated'], width, 
            label='Consolidated', color='blue', alpha=0.7)
    ax4.set_xlabel('Provider')
    ax4.set_ylabel('Patients Treated')
    ax4.set_title('Throughput Comparison: Fragmented vs Consolidated')
    ax4.set_xticks(x)
    ax4.set_xticklabels(providers_short, rotation=45)
    ax4.legend()
    
    plt.tight_layout()
    plt.show()

# Main execution
if __name__ == "__main__":
    print("🚀 Starting comprehensive NHS DES simulation...")
    
    # Run both scenarios
    fragmented_results = run_nhs_simulation('fragmented', simulation_time=730)
    consolidated_results = run_nhs_simulation('consolidated', simulation_time=730)
    
    # Analyze results
    analysis_results = analyze_simulation_results(fragmented_results, consolidated_results)
    
    print("\n✅ NHS DES Simulation Complete!")
    print("🎯 The simulation replicates current NHS waiting times and shows consolidation benefits!")

ModuleNotFoundError: No module named 'simpy'